# Question 2: Comparative Analysis of Ranking Algorithms

**Student:** Erfan Shahabi (810103166)

In [ ]:
import os
import sys

notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

print(f"Working directory: {os.getcwd()}")
print(f"Parent directory added: {parent_dir}")

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from src.question2 import ranking
from src.question2 import analysis
from src.question2 import stability
from src.question2 import visualization

print("All modules imported successfully!")

## Load Data

In [ ]:
G = nx.read_edgelist('../data/Wiki-Vote.txt', create_using=nx.DiGraph(), nodetype=int)

print(f"Nodes: {G.number_of_nodes()}")
print(f"Edges: {G.number_of_edges()}")
print(f"Directed: {G.is_directed()}")

print(f"\nAverage in-degree: {sum(dict(G.in_degree()).values())/G.number_of_nodes():.2f}")
print(f"Average out-degree: {sum(dict(G.out_degree()).values())/G.number_of_nodes():.2f}")

if not nx.is_weakly_connected(G):
    print("\nGraph is not weakly connected. Working with largest component...")
    largest_wcc = max(nx.weakly_connected_components(G), key=len)
    G = G.subgraph(largest_wcc).copy()
    print(f"Largest component: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Part (a): Ranking Comparison

### 1. Calculate HITS and PageRank

In [ ]:
print("Calculating HITS...")
hubs, authorities = ranking.calculate_hits(G)

print("Calculating PageRank (α=0.85)...")
pagerank_scores = ranking.calculate_pagerank(G, alpha=0.85)

print("\nTop 10 by HITS Authority:")
top_auth = sorted(authorities.items(), key=lambda x: x[1], reverse=True)[:10]
for node, score in top_auth:
    print(f"  Node {node}: {score:.6f}")

print("\nTop 10 by PageRank:")
top_pr = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)[:10]
for node, score in top_pr:
    print(f"  Node {node}: {score:.6f}")

### 2. Ranking Comparison & Visualization

In [ ]:
comparison = analysis.ranking_comparison(authorities, pagerank_scores)

print(f"Total nodes: {len(comparison)}")
print(f"\nRank difference statistics:")
print(comparison['rank_diff'].describe())

fig = visualization.plot_ranking_comparison_scatter(comparison)
plt.show()

print("\nTop 10 nodes with highest agreement (smallest rank difference):")
print(comparison.nsmallest(10, 'rank_diff')[['authority_rank', 'pagerank_rank', 'rank_diff']])

print("\nTop 10 nodes with highest divergence:")
print(comparison.nlargest(10, 'rank_diff')[['authority_rank', 'pagerank_rank', 'rank_diff']])

### 3. Divergence Analysis

In [ ]:
divergent_nodes = analysis.identify_divergent_nodes(comparison, threshold_percentile=95)

print(f"Identified {len(divergent_nodes)} divergent nodes (top 5% rank difference)\n")

print("Analyzing structural properties of divergent nodes...\n")

for idx, (node, row) in enumerate(divergent_nodes.head(5).iterrows()):
    in_deg = G.in_degree(node)
    out_deg = G.out_degree(node)
    
    print(f"Node {node}:")
    print(f"  Authority Rank: {row['authority_rank']:.0f}")
    print(f"  PageRank Rank: {row['pagerank_rank']:.0f}")
    print(f"  Rank Difference: {row['rank_diff']:.0f}")
    print(f"  In-degree: {in_deg}")
    print(f"  Out-degree: {out_deg}")
    print(f"  In/Out ratio: {in_deg/max(out_deg,1):.2f}")
    
    if row['authority_rank'] < row['pagerank_rank']:
        print(f"  HITS ranks higher: Strong incoming links from authoritative sources")
    else:
        print(f"  PageRank ranks higher: Better global connectivity and link structure")
    print()

## Part (b): Rank Stability Analysis

### 1. PageRank Sensitivity to α

In [ ]:
alpha_values = np.linspace(0.50, 0.85, 8)

print(f"Running PageRank sensitivity analysis for α values: {alpha_values}\n")

sensitivity_results = stability.pagerank_sensitivity_analysis(G, alpha_values)

print("Sensitivity analysis complete!")
print(f"Results shape: {sensitivity_results.shape}")
print(f"\nColumns: {list(sensitivity_results.columns)}")

### 2. Rank Trajectories Visualization

In [ ]:
trajectories = stability.analyze_rank_trajectories(sensitivity_results, top_n=10)

print("Top 10 nodes by average rank:")
print(trajectories)

fig = visualization.plot_rank_trajectories(trajectories)
plt.show()

### 3. Trajectory Interpretation

In [ ]:
rank_cols = [col for col in trajectories.columns if 'rank_' in col]

print("Rank Stability Analysis:\n")

for idx, row in trajectories.iterrows():
    node = row['node']
    ranks = [row[col] for col in rank_cols]
    
    rank_change = ranks[-1] - ranks[0]
    rank_std = np.std(ranks)
    
    print(f"Node {node}:")
    print(f"  Rank at α=0.50: {ranks[0]:.0f}")
    print(f"  Rank at α=0.85: {ranks[-1]:.0f}")
    print(f"  Total change: {rank_change:+.0f}")
    print(f"  Std deviation: {rank_std:.2f}")
    
    if abs(rank_change) < 5:
        print(f"  Stable: Rank largely unaffected by damping factor")
    elif rank_change > 0:
        print(f"  Declining: Benefits from local influence (low α)")
    else:
        print(f"  Improving: Benefits from distributed influence (high α)")
    print()

### 4. Sensitivity Heatmap

In [ ]:
fig = visualization.plot_sensitivity_heatmap(sensitivity_results, top_n=20)
plt.show()